In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform

In [2]:
# Functions

# Get panel masks

def get_panel_masks(geojson_path, ref_img_path):
    '''This function generates a mask that leaves only pixels inside 
    vine panels unmasked. 
    
    Inputs:
    geojson_path (str): path to a geojson file with the panel geometries. 
    Must be in same CRS as image, otherwise un-comment code section starting with "dst_crs".
    ref_img_path (str): path to the .tif file for the image
    
    Outputs:
    A list of masks, as Boolean arrays. True values are masked pixels.
    False values are unmasked pixels.
    '''
    with open(geojson_path, 'r') as f:
        panels = json.load(f)
        features = panels['features']
        features_ordered = sorted(features, key= lambda feature:(feature['properties']['row'],feature['properties']['panel'])) # ground truth order.
        panel_aois = [f['geometry'] for f in features_ordered]
    
    with rio.open(ref_img_path) as src:
        dst_crs = src.crs
        
        # geojson features (the field block geometries)
        # are often in WGS84
        # project these to the image coordinates
        wgs84 = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
        proj_panel_aois = [transform(project, shape(p)) for p in panel_aois]

        masks = [raster_geometry_mask(src, [p], crop=False)[0]
                 for p in proj_panel_aois]                       
    return masks

# Get reflectance df

def get_rfl(img_path, masks):
    '''
    Function to extract per=panel reflectance values for 4-band SkySat images.
    Calculates the mean reflectance values for the pixels in each panel.
    The output is formatted as a pandas dataframe.
    
    Inputs:
    img_path - abs. path to the image       
    masks - a list of masks that correspond to the panel aois. 
    '''
    
    with rio.open(img_path,'r') as img:
        def _get_stats(mask, panel_number):
            panel_blue = np.ma.mean(np.ma.array(img.read(1), mask=mask))/10000
            panel_green = np.ma.mean(np.ma.array(img.read(2), mask=mask))/10000
            panel_red = np.ma.mean(np.ma.array(img.read(3), mask=mask))/10000
            panel_nir = np.ma.mean(np.ma.array(img.read(4), mask=mask))/10000
            
            return {'acquired': img_path.split('/')[-1].split('_')[-7], # date from image filename
                    'plot': panel_number + 1,    #add one - panels are numbered starting at 1 in the 'real world'
                    'blue': panel_blue,
                    'green': panel_green,
                    'red': panel_red,
                    'nir': panel_nir}
        data = [_get_stats(m, i) for i, m in enumerate(masks)]  # sets the parameters for the nested function as the mask (m) and corresponding panel number (i). 
        df = pd.DataFrame(data)
    
    return df

In [3]:
# image directory containing co-registered SkySat scenes 

img_dir = '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2022/SkySat/'

# image pathnames
img_paths = glob.glob(img_dir+'2022*.tif')

# geojson with panel geometries

json_path = '/Users/kathleenkanaley/Desktop/grapes_in_space/map/qgis_layers/crit_panels_geojson.geojson'
json_path
img_paths

['/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2022/SkySat/20220720_153112_ssc4d2_0013_analytic_SR_clipped.tif',
 '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2022/SkySat/20220707_151445_ssc4d2_0006_analytic_SR_clipped.tif',
 '/Users/kathleenkanaley/Desktop/grapes_in_space/images_work/images_analysis_ready/2022/SkySat/20220622_160829_ssc15d2_0014_analytic_SR_clipped.tif']

In [4]:
# Get dataframes for all images in directory

master_df = pd.DataFrame()

for i in img_paths:
    masks = get_panel_masks(json_path, i)
    df = get_rfl(i, masks)
    master_df = pd.concat([master_df, df], axis=0)
    master_df = master_df.sort_values(by=['acquired', 'plot'])

master_df

,acquired,plot,blue,green,red,nir
0,20220622,1,0.058377,0.084667,0.086377,0.367607
1,20220622,2,0.058603,0.085176,0.088338,0.357583
2,20220622,3,0.059032,0.086039,0.087004,0.360371
3,20220622,4,0.057917,0.086170,0.088193,0.365127
4,20220622,5,0.059172,0.085890,0.089700,0.359786
...,...,...,...,...,...,...
315,20220720,316,0.032732,0.059432,0.057614,0.360093
316,20220720,317,0.032294,0.060229,0.058652,0.364500
317,20220720,318,0.034783,0.063197,0.060669,0.365607
318,20220720,319,0.031706,0.061275,0.058091,0.368478


In [5]:
# Append corresponding row and panel numbers for each plot 

panel_csv = pd.read_csv('/Users/kathleenkanaley/Desktop/grapes_in_space/map/qgis_layers/crit_panels.csv')
cols = panel_csv.iloc[:,1:] # adjust this according to the .csv
#cols
appended = pd.concat([master_df, cols],axis=1)
appended

,acquired,plot,blue,green,red,nir,row,panel
0,20220622,1,0.058377,0.084667,0.086377,0.367607,1,1
1,20220622,2,0.058603,0.085176,0.088338,0.357583,1,2
2,20220622,3,0.059032,0.086039,0.087004,0.360371,1,3
3,20220622,4,0.057917,0.086170,0.088193,0.365127,1,4
4,20220622,5,0.059172,0.085890,0.089700,0.359786,1,5
...,...,...,...,...,...,...,...,...
315,20220720,316,0.032732,0.059432,0.057614,0.360093,20,12
316,20220720,317,0.032294,0.060229,0.058652,0.364500,20,13
317,20220720,318,0.034783,0.063197,0.060669,0.365607,20,14
318,20220720,319,0.031706,0.061275,0.058091,0.368478,20,15


In [6]:
appended.acquired.unique()


array(['20220622', '20220707', '20220720'], dtype=object)

In [7]:
scout_csv = pd.read_csv('scout_2022_gdf.csv')
scout_csv.Date.unique()

array(['2022-06-22', '2022-06-29', '2022-07-06', '2022-07-14',
       '2022-07-20', '2022-07-27', '2022-08-02', '2022-08-09',
       '2022-08-29'], dtype=object)

In [8]:
# Match scout data to image data

scout_csv = pd.read_csv('scout_2022_gdf.csv')

# Define filtering and matching parameters

first_row = 1
last_row = 20

match_dates = ['2022-06-22','2022-07-06', '2022-07-20']
match_cols =['row', 'panel', 'Date']


# Filter and join datasets

img_filtered = appended[appended['row']>=first_row]
img_filtered = img_filtered[img_filtered['row']<=last_row]

scout_filtered = scout_csv[scout_csv['Row']>=first_row]
scout_filtered = scout_csv[scout_csv['Row']<=last_row]

scout_filtered = scout_filtered[scout_filtered['Date'].isin(match_dates)]
scout_filtered = scout_filtered.rename(columns={'Row':'row', 'Panel':'panel'})
scout_filtered[["row", "panel"]] = scout_filtered[["row", "panel"]].apply(pd.to_numeric)

# Create a 'Date' column in the image dataframe to match the scout date
def match_days(row):
    if row['acquired']=='20220622':
        return '2022-06-22'
    if row['acquired']=='20220707':
        return '2022-07-06'
    if row['acquired']=='20220720':
        return '2022-07-20'

img_filtered['Date'] = img_filtered.apply(lambda row: match_days(row), axis=1)

# Join and drop NANs + unneeded columns

joined = img_filtered.join(scout_filtered.set_index(match_cols), on=match_cols)
cleaned = joined.dropna(axis=0,subset=['DM_severity'])

cleaned.info()
# Uncomment below for masked rasters
#no_nullpix = cleaned.dropna(axis=0,subset=['blue', 'green', 'red', 'nir'])

#no_nullpix.info()
#joined = joined.rename(columns={'mean_sev_top_bot':'DM_severity'})
#cleaned = joined.dropna(axis=0,subset=['geometry', 'Treatment', 'leaf_avg', 'centroid'])
#cleaned = cleaned.drop(['Disease'], axis=1)




<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 318
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     720 non-null    object 
 1   plot         720 non-null    int64  
 2   blue         720 non-null    float64
 3   green        720 non-null    float64
 4   red          720 non-null    float64
 5   nir          720 non-null    float64
 6   row          720 non-null    int64  
 7   panel        720 non-null    int64  
 8   Date         720 non-null    object 
 9   Treatment    720 non-null    float64
 10  Block        720 non-null    object 
 11  PM_severity  720 non-null    float64
 12  DM_severity  720 non-null    float64
 13  total_dis    720 non-null    float64
 14  geometry     720 non-null    object 
 15  centroid     720 non-null    object 
dtypes: float64(8), int64(3), object(5)
memory usage: 95.6+ KB


In [9]:
# Export dataframe as .csv

output_fp = '/Users/kathleenkanaley/Desktop/grapes_in_space/stats_work/SkySat_2022/analysis_ready/'

#cleaned.to_csv(output_fp+'skysat_2022_nomask.csv', index=False)